Tutorial Followed from https://python.langchain.com/docs/modules/model_io/chat/function_calling/

In [3]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [3]:
# !pip install -qU langchain-openai

In [4]:
import api_key #my local file
import os

os.environ['OPENAI_API_KEY'] = api_key.OPENAI_API_KEY

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [42]:
query = "What is 3 * 12? Also, what is 11 + 49?"

In [43]:


t = llm_with_tools.invoke(query)
print(t)
t.tool_calls

content='' additional_kwargs={'tool_calls': [{'id': 'call_1XJuZJA9dEolY4Tm1EKeU7kX', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_28bmMQbGxMaRa6IliQ04w8Ya', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 112, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-4cec0147-d412-4972-9066-839ec1e24d88-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_1XJuZJA9dEolY4Tm1EKeU7kX'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_28bmMQbGxMaRa6IliQ04w8Ya'}]


[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_1XJuZJA9dEolY4Tm1EKeU7kX'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_28bmMQbGxMaRa6IliQ04w8Ya'}]

### Streaming

In [45]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_o7tFeMMudKjyHNbdLXQ9sfAk', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'add', 'args': '', 'id': 'call_d3lGQ5wxycH8tPiq5RVVPC3P', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


In [ ]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

In [ ]:
print(type(gathered.tool_call_chunks[0]["args"]))

###  partial parsing

In [ ]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

In [ ]:
print(type(gathered.tool_calls[0]["args"]))

### Passing tool outputs to model

In [7]:
from langchain_core.messages import HumanMessage, ToolMessage

In [46]:


messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3VgGaQHNymlWl2GjLq3cb08J', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_0ZwA1CI4RhIMclTWh8UkuLEA', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 112, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7faa4c13-5e3c-453d-a342-7614f603b66d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_3VgGaQHNymlWl2GjLq3cb08J'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_0ZwA1CI4RhIMclTWh8UkuLEA'}]),
 ToolMessage(content='36', tool_call_id='call_3VgGaQHNymlWl2GjLq3cb08J'),
 ToolMessage(content='60', tool_call_id='call_0ZwA1CI4RhIMclTWh8UkuLEA')]

In [ ]:
[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Dxs9hyOkP9z6uNpU4sqOgVC9', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_xA5iWbmwiDoJdpWhvY4tDu7T', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 112, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-648e8578-6c57-47ce-8501-4c764026baa9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_Dxs9hyOkP9z6uNpU4sqOgVC9'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_xA5iWbmwiDoJdpWhvY4tDu7T'}]),
 ToolMessage(content='36', tool_call_id='call_Dxs9hyOkP9z6uNpU4sqOgVC9'),
 ToolMessage(content='60', tool_call_id='call_xA5iWbmwiDoJdpWhvY4tDu7T')]

In [10]:
import pickle

In [13]:
pickle.dump(messages, open('messages.pkl', 'wb'))

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 = 36\n\n11 + 49 = 60', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 177, 'total_tokens': 193}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac217786-235a-4b8c-8de4-2226e155babc-0')

### Few-shot prompting

In [47]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_2Ots4e9jqtftnTXxin1NwLCk'},
 {'name': 'add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_rE1Dj09FZfmPHhztrj7qMeBa'}]

In [8]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

In [28]:
chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
# chain.invoke("Whats 119 times 8 minus 20").tool_calls
t = chain.invoke("Whats 119 times 8 minus 20")
print(t)
t.tool_calls

content='' additional_kwargs={'tool_calls': [{'id': 'call_akTJmNcHaddkrQfWMDACABUW', 'function': {'arguments': '{"a":119,"b":8}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 249, 'total_tokens': 266}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-e89ec2b3-e39b-4d22-83b5-29618b94021e-0' tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_akTJmNcHaddkrQfWMDACABUW'}]


[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_akTJmNcHaddkrQfWMDACABUW'}]

In [31]:
few_shot_prompt

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are bad at math but are an expert at using a calculator. \n\nUse past tool usage as an example of how to correctly use the tools.')), HumanMessage(content="What's the product of 317253 and 128472 plus four", name='example_user'), AIMessage(content='', name='example_assistant', tool_calls=[{'name': 'multiply', 'args': {'x': 317253, 'y': 128472}, 'id': '1'}]), ToolMessage(content='16505054784', tool_call_id='1'), AIMessage(content='', name='example_assistant', tool_calls=[{'name': 'add', 'args': {'x': 16505054784, 'y': 4}, 'id': '2'}]), ToolMessage(content='16505054788', tool_call_id='2'), AIMessage(content='The product of 317253 and 128472 plus four is 16505054788', name='example_assistant'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])

In [ ]:
ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are bad at math but are an expert at using a calculator. \n\nUse past tool usage as an example of how to correctly use the tools.')), HumanMessage(content="What's the product of 317253 and 128472 plus four", name='example_user'), AIMessage(content='', name='example_assistant', tool_calls=[{'name': 'multiply', 'args': {'x': 317253, 'y': 128472}, 'id': '1'}]), ToolMessage(content='16505054784', tool_call_id='1'), AIMessage(content='', name='example_assistant', tool_calls=[{'name': 'add', 'args': {'x': 16505054784, 'y': 4}, 'id': '2'}]), ToolMessage(content='16505054788', tool_call_id='2'), AIMessage(content='The product of 317253 and 128472 plus four is 16505054788', name='example_assistant'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])

In [22]:
dir(llm_with_tools)
# llm_with_tools.tools

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_up

In [36]:
chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools

In [37]:
t = chain.invoke("Whats 119 times 8 plus 20")
t

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WnAqhfk6miJDBrvT3KBj9bdb', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_YAGA6gXcO0sDwhEvKVgSSMNs', 'function': {'arguments': '{"a": 20, "b": 0}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 249, 'total_tokens': 298}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5d48a677-f2f6-42bb-8728-9af0800bb33d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_WnAqhfk6miJDBrvT3KBj9bdb'}, {'name': 'add', 'args': {'a': 20, 'b': 0}, 'id': 'call_YAGA6gXcO0sDwhEvKVgSSMNs'}])

In [32]:
query = "Whats 119 times 8 minus 20"

In [34]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
# # ai_msg = llm_with_tools.invoke(messages)
# I tried calling the above prompt with chain
ai_msg = chain.invoke(messages)
messages.append(ai_msg)
# messages
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    ai_msg = chain.invoke(messages)
    messages.append(ai_msg)
    messages
messages

[HumanMessage(content='Whats 119 times 8 minus 20'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4V5nHF6tLS9iwCFvnaQ54WBy', 'function': {'arguments': '{"a":119,"b":8}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 255, 'total_tokens': 272}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d3c079de-5030-44a4-b87d-fc5ab120288c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_4V5nHF6tLS9iwCFvnaQ54WBy'}]),
 ToolMessage(content='952', tool_call_id='call_4V5nHF6tLS9iwCFvnaQ54WBy'),
 AIMessage(content='The result of 119 times 8 minus 20 is 952.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 498, 'total_tokens': 514}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-a4090697-

This one did not work because the chain just tells us which first tool to call but it does not know to execute it, unless we manually do. And when we execute the first tool and expect the chain to give the next AI message with addition tool in tool calls, it just replies with a sentence with question in context. So, let's try this with agents since they can think by themselves on which action to take.

### Tool-Calling Agent

In [11]:
#prompt should contain ("placeholder", "{agent_scratchpad}"), so I'm rewriting the prompt here
examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Earlier we were using initialize agent to call the inbuilt tools or custom tools. If you implement tool calling, then it will be better to use the new create_tool_calling_agent function, it automatically calls the tools accordingly.

In [13]:
from langchain.agents import create_tool_calling_agent, AgentExecutor


agent = create_tool_calling_agent(llm_with_tools, tools, few_shot_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_ai_msg = agent_executor.invoke({"query" : "Whats 119 times 8 minus 20"})

print("Output:  ", agent_ai_msg)
# agent_ai_msg.tool_calls =====> ai_msg from the agent will not have .tool_calls
# agent_executor.invoke({"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 119, 'b': 8}`


952
Invoking: `subtract` with `{'a': 952, 'b': 20}`


subtract is not a valid tool, try one of [add, multiply].
Invoking: `add` with `{'a': 952, 'b': -20}`


932The result of 119 times 8 minus 20 is 932.

> Finished chain.
Output:   {'query': 'Whats 119 times 8 minus 20', 'output': 'The result of 119 times 8 minus 20 is 932.'}
